In [7]:
import pandas as pd
import numpy as np

from IPython.display import display
import datetime

In [8]:
matches_table = pd.read_csv('data/matches.csv')

In [9]:
display(matches_table)

,Date,Match Number,Stage,Player 1,Player 2,Score 1,Score 2
0,2023-11-18,1,Group,Evan Sooklal,Will Simpson,2,6
1,2023-11-18,2,Group,Paul Bartenfeld,Roman Ramirez,2,6
2,2023-11-18,3,Group,Roman Ramirez,Evan Sooklal,6,3
3,2023-11-18,4,Group,Will Simpson,Paul Bartenfeld,6,1
4,2023-11-18,5,Group,Paul Bartenfeld,Evan Sooklal,1,6
5,2023-11-18,6,Group,Will Simpson,Roman Ramirez,3,6
6,2023-11-18,7,Group,Will Simpson,Evan Sooklal,6,2
7,2023-11-18,8,Group,Roman Ramirez,Paul Bartenfeld,6,4
8,2023-11-18,9,Group,Evan Sooklal,Roman Ramirez,6,5
9,2023-11-18,10,Group,Paul Bartenfeld,Will Simpson,4,6


In [10]:
set_player1 = set(matches_table['Player 1'])
set_player2 = set(matches_table['Player 2'])
set_players = set_player1.union(set_player2)
num_players = len(set_players)
display(set_players)

{'Aaron Carter',
 'Evan Sooklal',
 'Paul Bartenfeld',
 'Roman Ramirez',
 'Will Simpson'}

In [11]:
# elo stuff

def expected_score(ratingA, ratingB):
    yrange = 12
    ymin = -6
    # game variables
    expected_cups = 2
    normal_elo_difference = 100

    return (yrange / (1 + np.power(expected_cups, (ratingB - ratingA) / normal_elo_difference))) + ymin

def rating_change(score, expected_score):
    # k-factor: determines how strongly a result affects the rating change
    # usually between 10 and 40, bit with a lot of games, we want to change it often
    K = 5
    return K * (score - expected_score)

# ELO INITIALIZATION
starting_elo = 1200.0
elo = dict.fromkeys(set_players, starting_elo)
elo_time = np.zeros([len(matches_table) + 1, num_players])
elo_time_table = pd.DataFrame(elo_time)
elo_time_table.columns = sorted(elo)
elo_time_table.replace(0, np.NaN, inplace=True)

prev_elo_time_table = elo_time_table.copy(deep=True)
d_elo_time_table = elo_time_table.copy(deep=True)
exp_elo_time_table = elo_time_table.copy(deep=True)
eaa_elo_time_table = elo_time_table.copy(deep=True)

elo_time_table.iloc[0,:] = starting_elo
prev_elo_time_table.iloc[0,:] = starting_elo
eaa_elo_time_table.iloc[0,:] = starting_elo


for (i, row) in matches_table.iterrows():
    elo_p1 = elo[row['Player 1']]
    elo_p2 = elo[row['Player 2']]

    prev_elo_time_table.loc[i + 1, row['Player 1']] = elo_p1
    prev_elo_time_table.loc[i + 1, row['Player 2']] = elo_p2
    
    win_prob_p1 = expected_score(elo_p1, elo_p2)
    win_prob_p2 = expected_score(elo_p2, elo_p1)
    
    exp_elo_time_table.loc[i + 1, row['Player 1']] = win_prob_p1
    exp_elo_time_table.loc[i + 1, row['Player 2']] = win_prob_p2

    eaa_elo_time_table.loc[i + 1, row['Player 1']] = expected_score(elo_p1, starting_elo)
    eaa_elo_time_table.loc[i + 1, row['Player 2']] = expected_score(elo_p2, starting_elo)

    rating_change_p1 = rating_change(row['Score 1'] - row['Score 2'], win_prob_p1)
    rating_change_p2 = rating_change(row['Score 2'] - row['Score 1'], win_prob_p2)

    d_elo_time_table.loc[i + 1, row['Player 1']] = rating_change_p1
    d_elo_time_table.loc[i + 1, row['Player 2']] = rating_change_p2

    
    elo[row['Player 1']] += rating_change_p1
    elo[row['Player 2']] += rating_change_p2

    elo_time_table.loc[i + 1, row['Player 1']] = elo[row['Player 1']]
    elo_time_table.loc[i + 1, row['Player 2']] = elo[row['Player 2']]

prev_elo_time_table = prev_elo_time_table.iloc[1:,:]
elo_time_table = elo_time_table.iloc[1:,:]
d_elo_time_table = d_elo_time_table.iloc[1:,:]
exp_elo_time_table = exp_elo_time_table.iloc[1:,:]
eaa_elo_time_table = eaa_elo_time_table.iloc[1:,:]

display(prev_elo_time_table)
display(d_elo_time_table)
display(elo_time_table)



,Aaron Carter,Evan Sooklal,Paul Bartenfeld,Roman Ramirez,Will Simpson
1,NaN,1200.000000,NaN,NaN,1200.000000
2,NaN,NaN,1200.000000,1200.000000,NaN
3,NaN,1180.000000,NaN,1220.000000,NaN
4,NaN,NaN,1180.000000,NaN,1220.000000
5,NaN,1169.132444,1159.132444,NaN,NaN
6,NaN,NaN,NaN,1230.867556,1240.867556
7,NaN,1193.093140,NaN,NaN,1224.828251
8,NaN,NaN,1135.171749,1246.906860,NaN
9,NaN,1176.379464,NaN,1245.837394,NaN
10,NaN,NaN,1136.241215,NaN,1241.541926


,Aaron Carter,Evan Sooklal,Paul Bartenfeld,Roman Ramirez,Will Simpson
1,NaN,-20.000000,NaN,NaN,20.000000
2,NaN,NaN,-20.000000,20.000000,NaN
3,NaN,-10.867556,NaN,10.867556,NaN
4,NaN,NaN,-20.867556,NaN,20.867556
5,NaN,23.960695,-23.960695,NaN,NaN
6,NaN,NaN,NaN,16.039305,-16.039305
7,NaN,-16.713675,NaN,NaN,16.713675
8,NaN,NaN,1.069466,-1.069466,NaN
9,NaN,12.085352,NaN,-12.085352,NaN
10,NaN,NaN,0.486854,NaN,-0.486854


,Aaron Carter,Evan Sooklal,Paul Bartenfeld,Roman Ramirez,Will Simpson
1,NaN,1180.000000,NaN,NaN,1220.000000
2,NaN,NaN,1180.000000,1220.000000,NaN
3,NaN,1169.132444,NaN,1230.867556,NaN
4,NaN,NaN,1159.132444,NaN,1240.867556
5,NaN,1193.093140,1135.171749,NaN,NaN
6,NaN,NaN,NaN,1246.906860,1224.828251
7,NaN,1176.379464,NaN,NaN,1241.541926
8,NaN,NaN,1136.241215,1245.837394,NaN
9,NaN,1188.464816,NaN,1233.752042,NaN
10,NaN,NaN,1136.728069,NaN,1241.055073


In [19]:
import plotly.graph_objects as go
import plotly.colors as pc

fig = go.Figure()

# adding player
ranking = 1
trace_colors = pc.qualitative.Bold # Light24
for (i, (player, current_rating)) in enumerate(sorted(elo.items(), key=lambda x:x[1], reverse=True)):
    fig.add_trace(go.Scatter(
        x=elo_time_table.index,
        y=elo_time_table[player],
        name=f'#{ranking} ({current_rating:.0f}) {player}',
        mode='lines+markers',
        connectgaps=True,
        text=[
            f"<br><b>Before-Game ELO: </b>{x[2]:.0f}<br><b>Expected Score Diff Against Opponent: </b>{x[0]:+.2f}<br><b>Expected Score Diff Against Average: </b>{x[1]:+.2f}<br><br><b>Change in ELO:</b> {x[3]:+.0f}<br>" for x in zip(
                exp_elo_time_table[player],
                eaa_elo_time_table[player],
                prev_elo_time_table[player],
                d_elo_time_table[player]
            )
        ],
        line=dict(
            shape='hv',
            color=trace_colors[i % len(trace_colors)]
        )
    ))
    ranking += 1

# adding highlighting by tournament
tournaments = list(sorted(set(matches_table['Date'])))
vrect_colors = ['green', 'red', 'yellow', 'blue', 'orange']
for (i, tourney) in enumerate(tournaments):
    fig.add_vrect(
        annotation_text=tourney,
        annotation_position="top left",
        x0=matches_table['Date'][matches_table['Date'] == tourney].index[0] + 0.5,
        x1=matches_table['Date'][matches_table['Date'] == tourney].index[-1] + 1.5,
        fillcolor=vrect_colors[i % len(vrect_colors)],
        opacity=0.1,
        line_width=0
    )

fig.update_layout(
    title=f'<b>Pokémon Showdown ELO Rating System by Roman Ramirez</b><br><i>Updated {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))} CT',
    xaxis_title='<b>Game Number</b>',
    yaxis_title='<b>ELO Rating</b>'
)

customdata = np.stack((
        list(matches_table['Player 1']),
        list(matches_table['Player 2']),
        list(matches_table['Score 1']),
        list(matches_table['Score 2']),
        matches_table['Date']
    ), axis=-1)
hovertemplate = (
    '<i>%{customdata[4]|%A, %B %d, %Y}, Game %{x}</i><br>' +
    '<b>%{fullData.name}</b><br><br>' + 
    '<b>%{customdata[0]} vs. %{customdata[1]}</b><br>' +
    '<b>Final Score:</b> %{customdata[2]}-%{customdata[3]}<br>' + 
    '%{text}' + 
    '<b>After-Game ELO:</b> %{y:,.0f}<br>' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate,
    opacity=0.8,
    legendgrouptitle_text='<b>#<i>Rank</i> (<i>Current ELO</i>) <i>Player</i></b>'
)
fig.show()
fig.write_html("index.html")